<a href="https://colab.research.google.com/github/rupaut98/img-det-benchmark/blob/detectron/dectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'


In [ ]:
import sys, os, distutils
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))


In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [4]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

This is to seperate every frames in the video

In [6]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg

import cv2
from google.colab.patches import cv2_imshow

im = cv2.VideoCapture('/content/video (1080p).mp4')
width = int(im.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(im.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = im.get(cv2.CAP_PROP_FPS)
num_frames = int(im.get(cv2.CAP_PROP_FRAME_COUNT))
video_writer = cv2.VideoWriter('out.mp4', fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)



In [7]:
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
v = VideoVisualizer(MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), ColorMode.IMAGE)

[11/07 18:24:50 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


model_final_f10217.pkl: 178MB [00:01, 178MB/s]                           


In [8]:
#uploading csv file
import io
import pandas as pd
from google.colab import files
uploaded = files.upload()


Saving dataframe.csv to dataframe.csv


In [9]:
import tqdm
import pandas as pd
header = ['Classes', 'Boxes']
df = pd.DataFrame(columns=header)
def runOnVideo(im, maxFrames):
    """ Runs the predictor on every frame in the video (unless maxFrames is given),
    and returns the frame with the predictions drawn.
    """
    global df
    readFrames = 0
    while True:
        hasFrame, frame = im.read()
        if not hasFrame:
            break

        # Get prediction results for this frame
        outputs = predictor(frame)


        classes = outputs['instances'].pred_classes.to('cpu')
        boxes = outputs['instances'].pred_boxes.to('cpu')

        metadata = predictor.metadata
        class_names = [metadata.thing_classes[class_id] for class_id in classes]

        frame_df = pd.DataFrame({'Classes': class_names, 'Boxes': boxes})
        df = pd.concat([df, frame_df], ignore_index=True)

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # Draw a visualization of the predictions using the video visualizer
        visualization = v.draw_instance_predictions(frame, outputs["instances"].to("cpu"))

        # Convert Matplotlib RGB format to OpenCV BGR format
        visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)

        yield visualization

        readFrames += 1
        if readFrames > maxFrames:
            break

# Create a cut-off for debugging
num_frames = 120

# Enumerate the frames of the video
for visualization in tqdm.tqdm(runOnVideo(im, num_frames), total=num_frames):

    # Write test image
    cv2.imwrite('POSE detectron2.png', visualization)

    # Write to video file
    video_writer.write(visualization)
df.to_csv('/content/dataframe.csv', index=False)

# Release resources
im.release()
video_writer.release()
cv2.destroyAllWindows()

  0%|          | 0/120 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
121it [02:51,  1.42s/it]


# New Section